In [ ]:
import pandas as pd
import telebot
import threading
import time
import logging
import os
from requests.exceptions import ConnectTimeout, ReadTimeout

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Replace 'YOUR_BOT_TOKEN' with your actual bot token
bot_token = "TG Bot Token"

# Initialize the bot with the correct token
bot = telebot.TeleBot(bot_token)

# Dictionary to store user preferences
user_preferences = {}

# Load course data from Excel
def load_course_data():
    try:
        if os.path.exists('course_data.xlsx'):
            df = pd.read_excel('course_data.xlsx')
            logging.info('Course data loaded successfully from Excel')
            return df
        else:
            logging.warning('course_data.xlsx file does not exist')
            return pd.DataFrame()  # Return an empty DataFrame if file does not exist
    except Exception as e:
        logging.error(f'Error loading course data from Excel: {e}')
        return pd.DataFrame()  # Return an empty DataFrame in case of error

# Function to handle /start command
@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.reply_to(message, "aman, Welcome to the NSU Course Notifier Bot! Please use /setpref <course_name> <section> to set your preferred course and section. And then use /check to check the availability")

# Function to handle /setpref command
@bot.message_handler(commands=['setpref'])
def handle_setpref(message):
    if len(message.text.split()) != 3:
        bot.reply_to(message, "Usage: /setpref <course_name> <section>")
        return

    user_id = message.from_user.id
    course_name = message.text.split()[1].upper().strip()  # Normalize input
    section = message.text.split()[2].strip()  # Normalize input
    user_preferences[user_id] = (course_name, section)
    bot.reply_to(message, f"Preference set: {course_name} Section {section}")

# Function to handle /check command
@bot.message_handler(commands=['check'])
def handle_check(message):
    user_id = message.from_user.id
    if user_id not in user_preferences:
        bot.reply_to(message, "You have not set any preferences. Use /setpref to set your preference.")
        return

    course_name, section = user_preferences[user_id]
    course_data = load_course_data()

    # Strip any extra whitespace from course_name and section, and convert course_name to upper case
    course_name = course_name.strip().upper()
    section = section.strip()

    result = course_data[(course_data['Course Name'].str.strip().str.upper() == course_name) & 
                         (course_data['Section'].astype(str).str.strip() == section)]

    if not result.empty:
        seats_available = result.iloc[0]['Seats Available']
        bot.reply_to(message, f"Course: {course_name} Section: {section} Seats Available: {seats_available}")
        # Set up a one-time alert for this course and section if seats are zero
        if int(seats_available) == 0:
            bot.send_message(user_id, "We will notify you if any seat available within 30 minutes, keep patience")
            set_seat_alert(user_id, course_name, section)
    else:
        bot.reply_to(message, f"No data found for Course: {course_name} Section: {section}")

# Store alerts for users who need seat notifications
seat_alerts = {}

def set_seat_alert(user_id, course_name, section):
    seat_alerts[user_id] = (course_name, section)
    logging.info(f'Seat alert set for user {user_id} for Course: {course_name} Section: {section}')
    send_repeated_notifications(user_id, course_name, section)

# Function to periodically check seat availability
def periodic_check():
    while True:
        try:
            course_data = load_course_data()
            for user_id, (course_name, section) in list(seat_alerts.items()):
                course_name = course_name.strip().upper()  # Ensure consistency
                section = section.strip()
                result = course_data[(course_data['Course Name'].str.strip().str.upper() == course_name) & 
                                     (course_data['Section'].astype(str).str.strip() == section)]
                if not result.empty:
                    seats_available = result.iloc[0]['Seats Available']
                    if int(seats_available) > 0:
                        bot.send_message(user_id, f"Seats available for Course: {course_name} Section: {section} - {seats_available}")
                        del seat_alerts[user_id]  # Remove alert after notifying the user
        except Exception as e:
            logging.error(f'Error in periodic check: {e}')
        time.sleep(300)  # Check every 5 minutes

# Function to send repeated notifications every 10 minutes for 30 minutes
def send_repeated_notifications(user_id, course_name, section):
    def notify_user():
        for _ in range(3):
            time.sleep(600)  # Wait for 10 minutes
            course_data = load_course_data()
            result = course_data[(course_data['Course Name'].str.strip().str.upper() == course_name) & 
                                 (course_data['Section'].astype(str).str.strip() == section)]
            if not result.empty:
                seats_available = result.iloc[0]['Seats Available']
                if int(seats_available) > 0:
                    bot.send_message(user_id, f"Seats available for Course: {course_name} Section: {section} - {seats_available}")
                    del seat_alerts[user_id]  # Stop notifications if seat becomes available
                    return
            if _ == 2:  # If it's the last iteration (after 30 minutes)
                bot.send_message(user_id, f"No seat available for Course: {course_name} Section: {section}. Please try again later.")
                del seat_alerts[user_id]  # Remove alert after the last notification

    threading.Thread(target=notify_user, daemon=True).start()

# Function to handle bot polling with retry logic
def start_polling():
    while True:
        try:
            bot.polling()
        except (ConnectTimeout, ReadTimeout) as e:
            logging.error(f'Polling error: {e}. Retrying in 15 seconds...')
            time.sleep(15)
        except Exception as e:
            logging.error(f'Unexpected error: {e}. Retrying in 15 seconds...')
            time.sleep(15)

# Start the periodic check in a separate thread
threading.Thread(target=periodic_check, daemon=True).start()

# Run the bot polling with error handling
start_polling()
